In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.backend import eval

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler

import tensorboard
import keras
from keras.utils import tf_utils
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
import sys, os, math, time, datetime, re

print("tf: ", tf.__version__)
print("tb: ", tensorboard.__version__)
print(os.getcwd())

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.config.set_visible_devices([], 'GPU')
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

snapshot = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Debugging with Tensorboard
logdir="logs/fit/lstm/" + snapshot
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

ISMOORE_DATASETS = True
path = '..//..//pythons//wisdm_script_and_data//wisdm_script_and_data//WISDM//wisdm_ar_v1.1_wt_overlap'
fileslist = [f for f in sorted(os.listdir(path)) if os.path.isfile(os.path.join(path, f))]
FILESNUMBER = 1
LSTMNUMBER  = 1

with open("../../params/params_har.txt") as f:
    hyperparams = dict([re.sub('['+' ,\n'+']','',x.replace(' .', '')).split('=') for x in f][1:-1])
hyperparams = dict([k, float(v)] for k, v in hyperparams.items())
hyperparams['testSize'] = 0.500
hyperparams['noUnits'] = 81
print(hyperparams)

def seperateValues(data, noInput, noOutput, isMoore=True):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        if isMoore:
            x_data_i = data[i].reshape(-1, noInput+noOutput)
            x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        else:
            x_data_i = data[i][:-1].reshape(-1, noInput)
            y_data_i = data[i][-1].reshape(-1, noOutput)
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

def fromBit_v0( b ) :
    return -0.9 if b == 0.0 else 0.9

def fromBit_v1( b ) :
    return 0 if b == 0.0 else 1

class customLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, batchSize, initialLearningRate, learningRateDecay, decayDurationFactor, numTrainingSteps, glorotScaleFactor=0.1, orthogonalScaleFactor=0.1, name=None):
        self.batchSize = batchSize
        self.initialLearningRate = initialLearningRate
        self.learningRateDecay = learningRateDecay
        self.decayDurationFactor = decayDurationFactor
        self.glorotScaleFactor = glorotScaleFactor
        self.orthogonalScaleFactor = orthogonalScaleFactor
        self.numTrainingSteps = numTrainingSteps
        self.name = name
        self.T = tf.constant(self.decayDurationFactor * (self.numTrainingSteps/self.batchSize), dtype=tf.float32, name="T")
        self.lr = self.initialLearningRate
    
    def __call__(self, step):
        self.step = tf.cast(step, tf.float32)
        self.lr = tf.cond(self.step > self.T, 
                           lambda: tf.constant(self.learningRateDecay * self.initialLearningRate, dtype=tf.float32),
                           lambda: self.initialLearningRate * (1.0 - (1.0 - self.learningRateDecay) * self.step / self.T)
                          )
        return self.lr

def lstm_wLRS_wtCMF_model(noInput, noOutput, timestep):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=hyperparams['noUnits'], input_shape=[timestep, noInput],
                   activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True,
                   recurrent_dropout=0.0, return_sequences=False, name='LSTM_layer'))
    model.add(tf.keras.layers.Dense(noInput+noOutput, activation='tanh', name='MLP_layer'))
    model.add(tf.keras.layers.Dense(noOutput))
    optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['batchSize'], hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                    beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_lstm")
    model.compile(optimizer=optimizer, loss = 'mse', run_eagerly=False)
    return model


def indexOfMax( xs ) :
    m = xs[ 0 ]
    k = 0
    for i in range( 0, xs.size ) :
        if xs[ i ] > m :
            m = xs[ i ]
            k = i
    return k
    
if __name__ == '__main__':
    print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
    for file_no in range(8):
        trainFile = f'train{file_no}.csv'
        valFile = f'val{file_no}.csv'
        print(trainFile, valFile)
        df_train = np.array(pd.read_csv(os.path.join(path, trainFile ), skiprows=1))
        df_val = np.array(pd.read_csv(os.path.join(path, valFile ), skiprows=1))

        timestep = 40
        print( "\nTrain and val shapes = ", df_train.shape, df_val.shape)

        with open(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=50.train.csv'), "r") as fp:
            [noIn, noOut] = [int(x) for x in fp.readline().replace('\n', '').split(',')]

        print('Step 2: Separating values and labels.')
        x_train, y_train = seperateValues(df_train, noIn, noOut, isMoore=ISMOORE_DATASETS)
        x_val, y_val = seperateValues(df_val, noIn, noOut, isMoore=ISMOORE_DATASETS)
        scaler = StandardScaler()
        x_train = (scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))).reshape(x_train.shape[0], timestep, noIn)
        x_val = (scaler.fit_transform(x_val.reshape(x_val.shape[0], -1))).reshape(x_val.shape[0], timestep, noIn)
        for i in range( y_train.shape[ 0 ] ) :
            for j in range( y_train.shape[ 1 ] ) :
                y_train[ i, j ] = fromBit_v0( y_train[ i, j ] )

        for i in range( y_val.shape[ 0 ] ) :
            for j in range( y_val.shape[ 1 ] ) :
                y_val[ i, j ] = fromBit_v0( y_val[ i, j ] )

        print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
        print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)

        model = lstm_wLRS_wtCMF_model(noIn, noOut, timestep=timestep)
        model_history = model.fit(
                            x_train, y_train,
                            batch_size=int(hyperparams['batchSize']),
                            verbose=1, # Suppress chatty output; use Tensorboard instead
                            # epochs=10,
                            epochs=int(hyperparams['numTrainingSteps']/(x_train.shape[0])),
                            validation_data=(x_val, y_val),
                            shuffle=True,
                            use_multiprocessing=False,
                            callbacks=[tensorboard_callback]
                        )
        y_pred = model.predict(x_val, verbose=1, batch_size=int(hyperparams['batchSize']))

        val_performance = model.evaluate(x_val, y_val, batch_size=int(hyperparams['batchSize']), verbose=1)
        print( f"\n{valFile}val_performance = {val_performance}\n")
        
        count = 0
        numCorrect = 0
        for i in range( y_pred.shape[ 0 ] ) :
            count += 1
            if indexOfMax( y_pred[ i ] ) == indexOfMax( y_val[ i ] ) :
                numCorrect += 1

        print( f"{valFile} val accuracy = {numCorrect / count}")

tf:  2.7.0
tb:  2.7.0
C:\Users\chaut\OneDrive - Heriot-Watt University\HIOF_Master\Master_Thesis\NewLSTM\Codes\tf_implementations\notebooks\rnnplus_for_har
{'batchSize': 4.0, 'numTrainingSteps': 320000.0, 'beta1': 0.974833, 'beta2': 0.99689, 'epsilon': 0.00388, 'decayDurationFactor': 0.979079, 'initialLearningRate': 0.002798, 'learningRateDecay': 0.001025, 'glorotScaleFactor': 0.1, 'orthogonalScaleFactor': 0.1, 'testSize': 0.5, 'noUnits': 81}
Step 1: Dividing the training and testing set with ratio 1:1 (50%).
train0.csv val0.csv

Train and val shapes =  (13769, 360) (13769, 360)
Step 2: Separating values and labels.
+ Training set:    (13769, 40, 3) (13769, 6) float64
+ Validating set:  (13769, 40, 3) (13769, 6) float64
Epoch 1/23
3443/3443 [==============================] - 66s 19ms/step - loss: 0.2042 - val_loss: 0.1524
Epoch 2/23
3443/3443 [==============================] - 64s 19ms/step - loss: 0.1212 - val_loss: 0.1081
Epoch 3/23
3443/3443 [==============================] - 65s 19

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.backend import eval

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler

import tensorboard
import keras
from keras.utils import tf_utils
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
import sys, os, math, time, datetime, re

print("tf: ", tf.__version__)
print("tb: ", tensorboard.__version__)
print(os.getcwd())

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.config.set_visible_devices([], 'GPU')
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

snapshot = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Debugging with Tensorboard
logdir="logs/fit/lstm/" + snapshot
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

ISMOORE_DATASETS = True
path = '..//..//pythons//wisdm_script_and_data//wisdm_script_and_data//WISDM//wisdm_ar_v1.1_wt_overlap'
fileslist = [f for f in sorted(os.listdir(path)) if os.path.isfile(os.path.join(path, f))]
FILESNUMBER = 1
LSTMNUMBER  = 1

with open("../../params/params_har.txt") as f:
    hyperparams = dict([re.sub('['+' ,\n'+']','',x.replace(' .', '')).split('=') for x in f][1:-1])
hyperparams = dict([k, float(v)] for k, v in hyperparams.items())
hyperparams['testSize'] = 0.500
hyperparams['noUnits'] = 81
print(hyperparams)

def seperateValues(data, noInput, noOutput, isMoore=True):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        if isMoore:
            x_data_i = data[i].reshape(-1, noInput+noOutput)
            x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        else:
            x_data_i = data[i][:-1].reshape(-1, noInput)
            y_data_i = data[i][-1].reshape(-1, noOutput)
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

def fromBit_v0( b ) :
    return -0.9 if b == 0.0 else 0.9

def fromBit_v1( b ) :
    return 0 if b == 0.0 else 1

class customLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, batchSize, initialLearningRate, learningRateDecay, decayDurationFactor, numTrainingSteps, glorotScaleFactor=0.1, orthogonalScaleFactor=0.1, name=None):
        self.batchSize = batchSize
        self.initialLearningRate = initialLearningRate
        self.learningRateDecay = learningRateDecay
        self.decayDurationFactor = decayDurationFactor
        self.glorotScaleFactor = glorotScaleFactor
        self.orthogonalScaleFactor = orthogonalScaleFactor
        self.numTrainingSteps = numTrainingSteps
        self.name = name
        self.T = tf.constant(self.decayDurationFactor * (self.numTrainingSteps/self.batchSize), dtype=tf.float32, name="T")
        self.lr = self.initialLearningRate
    
    def __call__(self, step):
        self.step = tf.cast(step, tf.float32)
        self.lr = tf.cond(self.step > self.T, 
                           lambda: tf.constant(self.learningRateDecay * self.initialLearningRate, dtype=tf.float32),
                           lambda: self.initialLearningRate * (1.0 - (1.0 - self.learningRateDecay) * self.step / self.T)
                          )
        return self.lr

def lstm_wLRS_wtCMF_model(noInput, noOutput, timestep):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=hyperparams['noUnits'], input_shape=[timestep, noInput],
                   activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True,
                   recurrent_dropout=0.0, return_sequences=False, name='LSTM_layer'))
    model.add(tf.keras.layers.Dense(noInput+noOutput, activation='tanh', name='MLP_layer'))
    model.add(tf.keras.layers.Dense(noOutput))
    optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['batchSize'], hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                    beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_lstm")
    model.compile(optimizer=optimizer, loss = 'mse', run_eagerly=False)
    return model


def indexOfMax( xs ) :
    m = xs[ 0 ]
    k = 0
    for i in range( 0, xs.size ) :
        if xs[ i ] > m :
            m = xs[ i ]
            k = i
    return k
    
if __name__ == '__main__':
    print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')

    for file_no in range(8):
        trainFile = f'train{file_no}.csv'
        valFile = f'val{file_no}.csv'
        print(trainFile, valFile)
        df_train = np.array(pd.read_csv(os.path.join(path, trainFile ), skiprows=1))
        df_val = np.array(pd.read_csv(os.path.join(path, valFile ), skiprows=1))

        timestep = 40
        print( "\nTrain and val shapes = ", df_train.shape, df_val.shape)

        with open(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=50.train.csv'), "r") as fp:
            [noIn, noOut] = [int(x) for x in fp.readline().replace('\n', '').split(',')]

        print('Step 2: Separating values and labels.')
        x_train, y_train = seperateValues(df_train, noIn, noOut, isMoore=ISMOORE_DATASETS)
        x_val, y_val = seperateValues(df_val, noIn, noOut, isMoore=ISMOORE_DATASETS)
        scaler = StandardScaler()
        x_train = (scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))).reshape(x_train.shape[0], timestep, noIn)
        x_val = (scaler.fit_transform(x_val.reshape(x_val.shape[0], -1))).reshape(x_val.shape[0], timestep, noIn)
        for i in range( y_train.shape[ 0 ] ) :
            for j in range( y_train.shape[ 1 ] ) :
                y_train[ i, j ] = fromBit_v1( y_train[ i, j ] )

        for i in range( y_val.shape[ 0 ] ) :
            for j in range( y_val.shape[ 1 ] ) :
                y_val[ i, j ] = fromBit_v1( y_val[ i, j ] )

        print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
        print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)

        model = lstm_wLRS_wtCMF_model(noIn, noOut, timestep=timestep)
        model_history = model.fit(
                            x_train, y_train,
                            batch_size=int(hyperparams['batchSize']),
                            verbose=1, # Suppress chatty output; use Tensorboard instead
                            epochs=int(hyperparams['numTrainingSteps']/(x_train.shape[0])),
                            validation_data=(x_val, y_val),
                            shuffle=True,
                            use_multiprocessing=False,
                            callbacks=[tensorboard_callback]
                        )
        y_pred = model.predict(x_val, verbose=1, batch_size=int(hyperparams['batchSize']))

        val_performance = model.evaluate(x_val, y_val, batch_size=int(hyperparams['batchSize']), verbose=1)
        print( f"\n{valFile}val_performance = {val_performance}\n")
        
        count = 0
        numCorrect = 0
        for i in range( y_pred.shape[ 0 ] ) :
            count += 1
            if indexOfMax( y_pred[ i ] ) == indexOfMax( y_val[ i ] ) :
                numCorrect += 1

        print( f"{valFile} val accuracy = {numCorrect / count}")

tf:  2.7.0
tb:  2.7.0
C:\Users\chaut\OneDrive - Heriot-Watt University\HIOF_Master\Master_Thesis\NewLSTM\Codes\tf_implementations\notebooks\rnnplus_for_har
{'batchSize': 4.0, 'numTrainingSteps': 320000.0, 'beta1': 0.974833, 'beta2': 0.99689, 'epsilon': 0.00388, 'decayDurationFactor': 0.979079, 'initialLearningRate': 0.002798, 'learningRateDecay': 0.001025, 'glorotScaleFactor': 0.1, 'orthogonalScaleFactor': 0.1, 'testSize': 0.5, 'noUnits': 81}
Step 1: Dividing the training and testing set with ratio 1:1 (50%).
train0.csv val0.csv

Train and val shapes =  (13769, 360) (13769, 360)
Step 2: Separating values and labels.
+ Training set:    (13769, 40, 3) (13769, 6) float64
+ Validating set:  (13769, 40, 3) (13769, 6) float64
Epoch 1/23
3443/3443 [==============================] - 61s 17ms/step - loss: 0.0658 - val_loss: 0.0494
Epoch 2/23
3443/3443 [==============================] - 56s 16ms/step - loss: 0.0412 - val_loss: 0.0389
Epoch 3/23
3443/3443 [==============================] - 56s 16

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.backend import eval

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler

import tensorboard
import keras
from keras.utils import tf_utils
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
import sys, os, math, time, datetime, re

print("tf: ", tf.__version__)
print("tb: ", tensorboard.__version__)
print(os.getcwd())

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.config.set_visible_devices([], 'GPU')
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

snapshot = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Debugging with Tensorboard
logdir="logs/fit/lstm/" + snapshot
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

ISMOORE_DATASETS = True
path = '..//..//pythons//wisdm_script_and_data//wisdm_script_and_data//WISDM//wisdm_ar_v1.1_wt_overlap'
fileslist = [f for f in sorted(os.listdir(path)) if os.path.isfile(os.path.join(path, f))]
FILESNUMBER = 1
LSTMNUMBER  = 1

with open("../../params/params_har.txt") as f:
    hyperparams = dict([re.sub('['+' ,\n'+']','',x.replace(' .', '')).split('=') for x in f][1:-1])
hyperparams = dict([k, float(v)] for k, v in hyperparams.items())
hyperparams['testSize'] = 0.500
hyperparams['noUnits'] = 81
print(hyperparams)

def seperateValues(data, noInput, noOutput, isMoore=True):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        if isMoore:
            x_data_i = data[i].reshape(-1, noInput+noOutput)
            x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        else:
            x_data_i = data[i][:-1].reshape(-1, noInput)
            y_data_i = data[i][-1].reshape(-1, noOutput)
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

def fromBit_v0( b ) :
    return -0.9 if b == 0.0 else 0.9

def fromBit_v1( b ) :
    return 0 if b == 0.0 else 1

class customLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, batchSize, initialLearningRate, learningRateDecay, decayDurationFactor, numTrainingSteps, glorotScaleFactor=0.1, orthogonalScaleFactor=0.1, name=None):
        self.batchSize = batchSize
        self.initialLearningRate = initialLearningRate
        self.learningRateDecay = learningRateDecay
        self.decayDurationFactor = decayDurationFactor
        self.glorotScaleFactor = glorotScaleFactor
        self.orthogonalScaleFactor = orthogonalScaleFactor
        self.numTrainingSteps = numTrainingSteps
        self.name = name
        self.T = tf.constant(self.decayDurationFactor * (self.numTrainingSteps/self.batchSize), dtype=tf.float32, name="T")
        self.lr = self.initialLearningRate
    
    def __call__(self, step):
        self.step = tf.cast(step, tf.float32)
        self.lr = tf.cond(self.step > self.T, 
                           lambda: tf.constant(self.learningRateDecay * self.initialLearningRate, dtype=tf.float32),
                           lambda: self.initialLearningRate * (1.0 - (1.0 - self.learningRateDecay) * self.step / self.T)
                          )
        return self.lr

def lstm_wLRS_wtCMF_model(noInput, noOutput, timestep):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=hyperparams['noUnits'], input_shape=[timestep, noInput],
                   activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True,
                   recurrent_dropout=0.0, return_sequences=False, name='LSTM_layer'))
    model.add(tf.keras.layers.Dense(noInput+noOutput, activation='tanh', name='MLP_layer'))
    model.add(tf.keras.layers.Dense(noOutput))
    optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['batchSize'], hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                    beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_lstm")
    model.compile(optimizer=optimizer, loss = 'mse', run_eagerly=False)
    return model


def indexOfMax( xs ) :
    m = xs[ 0 ]
    k = 0
    for i in range( 0, xs.size ) :
        if xs[ i ] > m :
            m = xs[ i ]
            k = i
    return k
    
if __name__ == '__main__':
    print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')

    for file_no in range(8):
        trainFile = f'train{file_no}.csv'
        valFile = f'val{file_no}.csv'
        print(trainFile, valFile)
        df_train = np.array(pd.read_csv(os.path.join(path, trainFile ), skiprows=1))
        df_val = np.array(pd.read_csv(os.path.join(path, valFile ), skiprows=1))

        timestep = 40
        print( "\nTrain and val shapes = ", df_train.shape, df_val.shape)

        with open(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=50.train.csv'), "r") as fp:
            [noIn, noOut] = [int(x) for x in fp.readline().replace('\n', '').split(',')]

        print('Step 2: Separating values and labels.')
        x_train, y_train = seperateValues(df_train, noIn, noOut, isMoore=ISMOORE_DATASETS)
        x_val, y_val = seperateValues(df_val, noIn, noOut, isMoore=ISMOORE_DATASETS)
        scaler = StandardScaler()
        x_train = (scaler.fit_transform(x_train.reshape(x_train.shape[0], -1))).reshape(x_train.shape[0], timestep, noIn)
        x_val = (scaler.fit_transform(x_val.reshape(x_val.shape[0], -1))).reshape(x_val.shape[0], timestep, noIn)
        for i in range( y_train.shape[ 0 ] ) :
            for j in range( y_train.shape[ 1 ] ) :
                y_train[ i, j ] = fromBit_v1( y_train[ i, j ] )

        for i in range( y_val.shape[ 0 ] ) :
            for j in range( y_val.shape[ 1 ] ) :
                y_val[ i, j ] = fromBit_v1( y_val[ i, j ] )

        print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
        print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)

        model = lstm_wLRS_wtCMF_model(noIn, noOut, timestep=timestep)
        model_history = model.fit(
                            x_train, y_train,
                            batch_size=int(hyperparams['batchSize']),
                            verbose=1, # Suppress chatty output; use Tensorboard instead
                            epochs=int(hyperparams['numTrainingSteps']/(x_train.shape[0])),
                            validation_data=(x_val, y_val),
                            shuffle=True,
                            use_multiprocessing=False,
                            callbacks=[tensorboard_callback]
                        )
        y_pred = model.predict(x_val, verbose=1, batch_size=int(hyperparams['batchSize']))

        val_performance = model.evaluate(x_val, y_val, batch_size=int(hyperparams['batchSize']), verbose=1)
        print( f"\n{valFile}val_performance = {val_performance}\n")
        
        count = 0
        numCorrect = 0
        for i in range( y_pred.shape[ 0 ] ) :
            count += 1
            if indexOfMax( y_pred[ i ] ) == indexOfMax( y_val[ i ] ) :
                numCorrect += 1

        print( f"{valFile} val accuracy = {numCorrect / count}")